In [20]:
# imports
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import time
import boto3
from urllib.request import urlopen
import json
import ffmpeg

In [21]:
# AWS Constants
S3_BUCKET_NAME = 'inflectionbucket'
AWS_PROFILE_NAME = 'patwang123'
AWS_REGION = 'us-east-1'

In [23]:
# Other constants
SUPPORTED_FORMATS = ['.wav', '.mp3']

recording_uri = lambda u: f"recordings/{u}"
data_uri = lambda u: f"data/{u}"
transcription_uri = lambda u: f"transcriptions/{u}"
s3_recording_uri = lambda u: f"s3://{S3_BUCKET_NAME}/{recording_uri(u)}"

In [24]:
# Create folders if does not already exist
# Recording -- our voices that are being converted to text and then to polly
# Transcriptions -- the transcripts
# Data -- the voice generated results from AWS
cwd = os.getcwd()
for folder in ['data', 'recordings', 'transcriptions']:
    path = os.path.join(cwd, folder)
    if not os.path.exists(path):
        os.makedirs(path)

In [25]:
# Start s3, Polly, and Transcribe AWS sessions
session = boto3.Session(profile_name=AWS_PROFILE_NAME, region_name=AWS_REGION)
s3 = session.client('s3')
polly = session.client("polly")
transcribe = session.client('transcribe')

In [26]:
# Create bucket if does not exist yet
try:
    s3.create_bucket(Bucket=S3_BUCKET_NAME)
except:
    pass

In [52]:
# Converts recordings to wav if mp3
items = set(os.listdir(os.path.join(os.getcwd(), 'recordings')))
for f in items:
    if f[-4:] != '.wav':
        if os.path.splitext(f)[0] + '.wav' not in items:
            save_path = os.path.join(os.getcwd(), 'recordings', f)
            stream = ffmpeg.input(save_path)
            stream = ffmpeg.output(stream, os.path.join(os.getcwd(), 'recordings', os.path.splitext(f)[0] + '.wav'))
            ffmpeg.run(stream)
        os.remove(save_path)
        
# ffmpeg -i obama.wav -ac 1 obamaShit.wav
# ffmpeg -i obamaShit.wav -ar 22050 obamaShittier.wav

In [51]:
# Uploads all files into s3
for filename in filenames:
    with closing(open(recording_uri(filename), 'rb')) as f:
        p = s3.upload_fileobj(f, S3_BUCKET_NAME, recording_uri(filename))
        
# list of file names that should be located in the recordings/ to process
filenames = os.listdir(os.path.join(cwd, 'recordings'))

# to check if we have already processed it, as if it is already in data, there is no need to process it again
data_done = set([os.path.splitext(f)[0] for f in os.listdir(os.path.join(cwd, 'data'))])

for filename in filenames:
    job_name = filename
    job_uri = s3_recording_uri(filename) # s3://DOC-EXAMPLE-BUCKET1/key-prefix/file.file-extension"

    print(f"Starting transcription job for {filename}")
    if filename[-4:] not in SUPPORTED_FORMATS:
        print(f"File does not end with one of {SUPPORTED_FORMATS}, skipping...")
        continue
    if os.path.splitext(filename)[0] in data_done:
        print("Data has already been processed and is in the data folder, skipping...")
        continue
    try:
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat=filename[-3:],
            LanguageCode='en-US'
            )
    except:
        transcribe.delete_transcription_job(TranscriptionJobName=job_name)
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat=filename[-3:],
            LanguageCode='en-US'
        )

    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        print(f"Transcription job '{job_name}' is not ready yet...")
        time.sleep(5)

    assert status['ResponseMetadata']['HTTPStatusCode'] == 200

    # Get location of data from transcription job and read in json
    transcribe_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
    opened = urlopen(transcribe_url)
    data_json = json.loads(opened.read())

    # Delete transcription job to clean up AWS or whatever
    transcribe.delete_transcription_job(TranscriptionJobName=job_name)

    # Save file to transcription folder
    with closing(open(transcription_uri(f"{os.path.splitext(filename)[0]}.json"), 'w')) as f:
        json.dump(data_json, f)

    # Using polly to synthesize speech based on transcript
    try:
        # Request speech synthesis
        response = polly.synthesize_speech(Text=data_json['results']['transcripts'][0]['transcript'],
                                           OutputFormat="mp3",
                                           VoiceId="Joanna")
    except (BotoCoreError, ClientError) as error:
        # The service returned an error, exit gracefully
        print(error)
        continue

    # Save polly response to data
    if 'AudioStream' in response:
        with closing(response["AudioStream"]) as stream:
            save_path = os.path.join(os.getcwd(), 'data', filename + "TEMP")
            file = open(save_path, 'wb')
            file.write(stream.read())
        stream = ffmpeg.input(save_path)
        stream = ffmpeg.output(stream, os.path.join(os.getcwd(), 'data', os.path.splitext(filename)[0] + '.wav'))
        ffmpeg.run(stream)
        os.remove(save_path)


Starting transcription job for obama.wav
Data has already been processed and is in the data folder, skipping...
Starting transcription job for obamaShittier.wav
Transcription job 'obamaShittier.wav' is not ready yet...
Transcription job 'obamaShittier.wav' is not ready yet...
Transcription job 'obamaShittier.wav' is not ready yet...
Transcription job 'obamaShittier.wav' is not ready yet...
Starting transcription job for obamaShit.wav
Data has already been processed and is in the data folder, skipping...


ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

In [30]:
print('yay')

yay
